# Data preprocessing

Notebook for preprocessing overtrading datasets.


## Concepts

- **Session**: one fixed-time window (e.g., each 15-minute window is a session).
- **Core window**: same as session in this setup.
- **Overtrading indicators**: optional global summaries across all windows (for context only).

This notebook builds one **training row per window**.

**Note:** each row corresponds to a single window and includes both the core-window
features and the per-window overtrading indicators.


In [ ]:
import pandas as pd
from pathlib import Path

# Load mock behaviors datasets
base_dir = Path("../../../")
data_dir = base_dir / "mock_behaviors"
if not data_dir.exists():
    # Backward-compat for existing folder name
    data_dir = base_dir / "mock_behaviours"

files = sorted(data_dir.glob("*_example.csv"))
files


In [ ]:
# Load one dataset
sample_path = files[0] if files else None
sample_path


In [ ]:
import math


def load_trades(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.sort_values("timestamp").reset_index(drop=True)
    df["notional"] = df["quantity"] * df["entry_price"]
    return df


df = load_trades(sample_path) if sample_path else pd.DataFrame()
df.head()


In [ ]:
def compute_core_window_vector(
    df: pd.DataFrame,
    window_minutes: int = 15,
    stride_minutes: int = 5,
    eps: float = 1e-9,
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()

    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.sort_values("timestamp")

    start = df["timestamp"].min().floor("min")
    end = df["timestamp"].max().ceil("min")

    window_starts = pd.date_range(start=start, end=end, freq=f"{stride_minutes}min", tz="UTC")

    rows = []
    for ws in window_starts:
        we = ws + pd.Timedelta(minutes=window_minutes)
        w = df[(df["timestamp"] >= ws) & (df["timestamp"] < we)].copy()

        n_trades = len(w)
        trade_rate_per_min = n_trades / window_minutes

        if n_trades >= 2:
            w["dt_prev_sec"] = w["timestamp"].diff().dt.total_seconds()
            gaps = w["dt_prev_sec"].dropna()
            median_gap_sec = float(gaps.median())
            mean_gap_sec = float(gaps.mean())
            gap_cv = float(gaps.std() / (gaps.mean() + eps))
            burst_frac = float((gaps <= 60).mean())
        else:
            median_gap_sec = float("nan")
            mean_gap_sec = float("nan")
            gap_cv = float("nan")
            burst_frac = float("nan")

        n_assets = int(w["asset"].nunique()) if n_trades else 0
        if n_trades:
            top_asset_share = float(w["asset"].value_counts(normalize=True).iloc[0])
        else:
            top_asset_share = float("nan")

        if n_trades >= 2:
            asset_switch_rate = float((w["asset"].shift() != w["asset"]).mean())
        else:
            asset_switch_rate = float("nan")

        notional_sum = float(w["notional"].sum()) if n_trades else 0.0
        notional_mean = float(w["notional"].mean()) if n_trades else float("nan")
        notional_std = float(w["notional"].std()) if n_trades else float("nan")

        if n_trades and "balance" in w.columns:
            window_start_balance = float(w["balance"].iloc[0])
            turnover = notional_sum / (window_start_balance + eps)
            dd_max = (float(w["balance"].min()) - window_start_balance) / (window_start_balance + eps)
        else:
            window_start_balance = float("nan")
            turnover = float("nan")
            dd_max = float("nan")

        pnl_sum = float(w["profit_loss"].sum()) if n_trades else 0.0
        pnl_mean = float(w["profit_loss"].mean()) if n_trades else float("nan")
        pnl_std = float(w["profit_loss"].std()) if n_trades else float("nan")

        if n_trades:
            wins = w[w["profit_loss"] > 0]
            losses = w[w["profit_loss"] < 0]
            win_rate = float((w["profit_loss"] > 0).mean())
            avg_gain = float(wins["profit_loss"].mean()) if len(wins) else float("nan")
            avg_loss_abs = float(losses["profit_loss"].abs().mean()) if len(losses) else float("nan")
            payoff_ratio = avg_gain / (avg_loss_abs + eps) if not math.isnan(avg_gain) else float("nan")
            p90 = float(w["profit_loss"].quantile(0.90))
            p10 = float(w["profit_loss"].quantile(0.10))
            p50 = float(w["profit_loss"].quantile(0.50))
            pnl_skew_proxy = (p90 + p10) / (abs(p50) + eps)
        else:
            win_rate = float("nan")
            avg_gain = float("nan")
            avg_loss_abs = float("nan")
            payoff_ratio = float("nan")
            pnl_skew_proxy = float("nan")

        rows.append(
            {
                "window_start": ws,
                "window_end": we,
                "n_trades": n_trades,
                "trade_rate_per_min": trade_rate_per_min,
                "median_gap_sec": median_gap_sec,
                "mean_gap_sec": mean_gap_sec,
                "gap_cv": gap_cv,
                "burst_frac": burst_frac,
                "n_assets": n_assets,
                "top_asset_share": top_asset_share,
                "asset_switch_rate": asset_switch_rate,
                "notional_sum": notional_sum,
                "notional_mean": notional_mean,
                "notional_std": notional_std,
                "turnover": turnover,
                "pnl_sum": pnl_sum,
                "pnl_mean": pnl_mean,
                "pnl_std": pnl_std,
                "win_rate": win_rate,
                "avg_gain": avg_gain,
                "avg_loss_abs": avg_loss_abs,
                "payoff_ratio": payoff_ratio,
                "pnl_skew_proxy": pnl_skew_proxy,
                "dd_max": dd_max,
                "window_start_balance": window_start_balance,
            }
        )

    return pd.DataFrame(rows)


core_windows = compute_core_window_vector(df, window_minutes=15, stride_minutes=5)
core_windows.head()


## Training dataframe (one row per window)

Each window is treated as a session, so the training dataframe is simply the
core window features with a `session_id` per window.


In [ ]:
if core_windows.empty:
    training_df = pd.DataFrame()
else:
    training_df = core_windows.copy()
    training_df["session_id"] = training_df["window_start"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

training_df.head()


In [ ]:
def add_overtrading_indicators_per_window(
    core_windows: pd.DataFrame,
    window_minutes: int = 15,
) -> pd.DataFrame:
    """Add overtrading indicators per window (session)."""
    if core_windows.empty:
        return pd.DataFrame()

    dfw = core_windows.copy().sort_values("window_start").reset_index(drop=True)

    # Global threshold used to label each window
    p90_trade_rate = float(dfw["trade_rate_per_min"].quantile(0.90))
    dfw["trade_rate_gt_p90"] = dfw["trade_rate_per_min"] > p90_trade_rate
    dfw["p90_trade_rate_global"] = p90_trade_rate

    # Per-window turnover per hour
    if "turnover" in dfw.columns:
        dfw["turnover_per_hour"] = dfw["turnover"] / (window_minutes / 60.0)
    else:
        dfw["turnover_per_hour"] = float("nan")

    # Hot streak length up to this window
    streak = 0
    streaks = []
    for is_hot in dfw["trade_rate_gt_p90"].tolist():
        if is_hot:
            streak += 1
        else:
            streak = 0
        streaks.append(streak)
    dfw["hot_streak_len"] = streaks

    return dfw


training_df = add_overtrading_indicators_per_window(training_df, window_minutes=15)
training_df.head()


In [ ]:
# Save training data to CSV in this folder
output_path = Path("data_preprocessing_training.csv")
if not training_df.empty:
    training_df.to_csv(output_path, index=False)
output_path
